In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt

from sequence_jacobian import het, simple, create_model              # functions
from sequence_jacobian import interpolate, grids, misc, estimation   # modules

def household_guess(a_grid,e_grid,r,w,gamma):
    wel = (1+r)*a_grid[np.newaxis,:] + w*e_grid[:,np.newaxis]
    V_prime = (1+r)*(wel/2)**(-gamma)
    return V_prime

# Important: V_prime MUST be named v_prime_p in the argument of the function! Otherwise, raises error

@het(exogenous = 'Pi',policy = 'a', backward = 'V_prime', backward_init=household_guess)
def household(V_prime_p,a_grid,e_grid,r,w,beta,gamma):
    c_prime = (beta*V_prime_p)**(-1/gamma)
    new_grid = c_prime + a_grid
    wel = (1+r)*a_grid[np.newaxis,:] + w*e_grid[:,np.newaxis]

    a = interpolate.interpolate_y(new_grid,wel,a_grid)
    misc.setmin(a,a_grid[0])
    c = wel - a
    V_prime= (1+r)*c**(-gamma)

    return V_prime,a,c

print(household)
print(f'Inputs: {household.inputs}')
print(f'Macro outputs: {household.outputs}')
print(f'Micro outputs: {household.internals}')

In [22]:
def make_grid(rho_e, sd_e, nE, amin, amax, nA):
    e_grid, _, Pi = grids.markov_rouwenhorst(rho=rho_e, sigma=sd_e, N=nE)
    a_grid = grids.agrid(amin=amin, amax=amax, n=nA)
    return e_grid, Pi, a_grid


household_ext = household.add_hetinputs([make_grid])

print(household_ext)
print(f'Inputs: {household_ext.inputs}')

<HetBlock 'household'>
Inputs: ['a_grid', 'e_grid', 'r', 'w', 'beta', 'gamma', 'backward_init', 'Pi']
Macro outputs: ['A', 'C']
Micro outputs: ['D', 'Dbeg', 'Pi', 'V_prime', 'a', 'c']


In [23]:
@simple
def firm(K, L, Z, alpha, delta):
    r = alpha * Z * (K(-1) / L) ** (alpha-1) - delta
    w = (1 - alpha) * Z * (K(-1) / L) ** alpha
    Y = Z * K(-1) ** alpha * L ** (1 - alpha)
    return r, w, Y


@simple
def mkt_clearing(K, A, Y, C, delta):
    asset_mkt = A - K
    goods_mkt = Y - C - delta * K
    return asset_mkt, goods_mkt

ks = create_model([household_ext, firm, mkt_clearing], name="Krusell-Smith")
print(ks.inputs)

<HetBlock 'household' with hetinput 'make_grid'>
Inputs: ['r', 'w', 'beta', 'gamma', 'backward_init', 'rho_e', 'sd_e', 'nE', 'amin', 'amax', 'nA']


In [24]:

calibration = {'eis': 1, 'delta': 0.025, 'alpha': 0.11, 'rho_e': 0.966, 'sd_e': 0.5, 'L': 1.0,
               'nE': 7, 'nA': 500, 'amin': 0, 'amax': 200}
unknowns_ss = {'beta': 0.98, 'Z': 0.85, 'K': 3.}
targets_ss = {'r': 0.01, 'Y': 1., 'asset_mkt': 0.}

ss = ks.solve_steady_state(calibration, unknowns_ss, targets_ss, solver='hybr')

print(ss)

['beta', 'gamma', 'backward_init', 'rho_e', 'sd_e', 'nE', 'amin', 'amax', 'nA', 'K', 'L', 'Z', 'alpha', 'delta']


In [25]:
calibration = {'gamma': 1, 'delta': 0.025, 'alpha': 0.11, 'rho_e': 0.966, 'sd_e': 0.5, 'L': 1.0,
               'nE': 7, 'nA': 500, 'amin': 0, 'amax': 200}
unknowns_ss = {'beta': 0.98, 'Z': 0.85, 'K': 3.}
targets_ss = {'r': 0.01, 'Y': 1., 'asset_mkt': 0.}

ss = ks.solve_steady_state(calibration, unknowns_ss, targets_ss, solver='hybr')
print(ss)

KeyError: 'backward_init'

In [21]:
e_grid, Pi, a_grid = make_grid(0.9,1,3,0.1,5,100)
household_guess(a_grid,e_grid,0.01,1,1)

array([[7.95303782, 7.60829878, 7.28454141, 6.97999921, 6.69309795,
        6.42243021, 6.16673386, 5.92487375, 5.69582613, 5.47866532,
        5.2725522 , 5.07672434, 4.89048744, 4.71320784, 4.54430607,
        4.38325116, 4.22955566, 4.08277127, 3.942485  , 3.80831572,
        3.67991117, 3.55694529, 3.43911576, 3.32614197, 3.217763  ,
        3.11373602, 3.01383466, 2.9178477 , 2.82557779, 2.73684033,
        2.65146249, 2.56928222, 2.49014749, 2.41391547, 2.3404519 ,
        2.26963039, 2.20133192, 2.13544426, 2.07186151, 2.01048366,
        1.95121621, 1.89396972, 1.83865958, 1.78520561, 1.7335318 ,
        1.68356605, 1.63523992, 1.58848837, 1.54324961, 1.49946485,
        1.45707815, 1.41603623, 1.37628832, 1.33778602, 1.30048318,
        1.26433573, 1.22930159, 1.19534057, 1.16241425, 1.13048589,
        1.09952032, 1.0694839 , 1.04034441, 1.01207096, 0.98463397,
        0.95800507, 0.93215703, 0.90706374, 0.88270012, 0.8590421 ,
        0.83606655, 0.81375122, 0.79207475, 0.77